In [230]:
function create_LHS_matrix(z,λ,Q,G,h,A=nothing)
    if A == nothing || size(A)[1] == 0
        return [Q                G';
                Diagonal(λ)*G    Diagonal(G*z - h)]
    else
        @assert size(A)[2] == size(G)[2]
        p, n = size(A)
        m    = size(G)[1]
        return [Q                G'                  A';  
                Diagonal(λ)*G    Diagonal(G*z - h)   zeros(m,p);
                A                zeros(p,m)          zeros(p,p)]
    end
end

create_LHS_matrix (generic function with 2 methods)

In [231]:
function create_RHS_matrix(z,dQ,dq,λ,dG,dh,ν=nothing,dA=nothing,db=nothing)
    if dA == nothing || size(dA)[1] == 0
        return -[dQ*z + dq + dG'*λ;
                 Diagonal(λ)*(dG*z - dh)]
    else
        return -[dQ*z + dq + dG'*λ + dA'*ν;
                 Diagonal(λ)*(dG*z - dh);
                 dA*z - db]
    end
end

create_RHS_matrix (generic function with 4 methods)

In [11]:
create_RHS_matrix(z,zeros(2,2),zeros(2,1),λ,zeros(2,2),ones(2,1))

4×1 Array{Float64,2}:
 -0.0 
 -0.0 
  0.25
  0.0 

In [244]:
coefficient(t::MOI.ScalarAffineTerm) = t.coefficient

"""
    Return problem parameters as matrices and other info
"""
function get_problem_data(model::MOI.AbstractOptimizer)
    var_idx = MOI.get(model, MOI.ListOfVariableIndices())
    nz = size(var_idx)[1]

    # handle inequality constraints
    ineq_con_idx = MOI.get(
                        model, 
                        MOI.ListOfConstraintIndices{
                            MOI.ScalarAffineFunction{Float64}, 
                            MOI.LessThan{Float64}
                        }())
    nineq = size(ineq_con_idx)[1]

    G = zeros(nineq, nz)
    h = zeros(nineq)

    for i in 1:nineq
        con = ineq_con_idx[i]

        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)

        G[i, :] = coefficient.(func.terms)'
        h[i] = set.upper - func.constant
    end
    
    # handle equality constraints
    eq_con_idx   = MOI.get(
                        model, 
                        MOI.ListOfConstraintIndices{
                            MOI.ScalarAffineFunction{Float64}, 
                            MOI.EqualTo{Float64}
                        }())
    neq   = size(eq_con_idx)[1]
    
    if neq > 0
        A = zeros(neq, nz)
        b = zeros(neq)
        
        for i in 1:neq
            con = eq_con_idx[i]

            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)

            A[i, :] = coefficient.(func.terms)'
            b[i]    = set.value - func.constant
        end
    else
        A = nothing
        b = nothing
    end
    
    # handle objective
    # works both for affine and quadratic objective functions
    objective_function = MOI.get(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}())
    Q = zeros(nz,nz)
    
    if typeof(objective_function) == MathOptInterface.ScalarAffineFunction{Float64}
        q = coefficient.(objective_function.terms)
    elseif typeof(objective_function) == MathOptInterface.ScalarQuadraticFunction{Float64}
        @assert size(objective_function.quadratic_terms)[1] == (nz*(nz+1))/2    
        
        var_to_id = Dict(var_idx .=> 1:nz)
        
        for quad in objective_function.quadratic_terms
            i = var_to_id[quad.variable_index_1]
            j = var_to_id[quad.variable_index_2]
            Q[i,j] = quad.coefficient
            Q[j,i] = quad.coefficient
        end
        
        q = coefficient.(objective_function.affine_terms)
    end
    
    return Q, q, G, h, A, b, nz, var_idx, nineq, ineq_con_idx, neq, eq_con_idx
end

get_problem_data

In [245]:
function DiffOpt(_model::MOI.AbstractOptimizer)
    model = deepcopy(_model)
    
    Q, q, G, h, A, b, nz, var_idx, nineq, ineq_con_idx, neq, eq_con_idx = get_problem_data(model)
    
    z = zeros(0) # solution
    λ = zeros(0) # lagrangian variables
    ν = zeros(0)
    
    
    """
        Solving the convex optimization problem in forward pass
    """
    function forward()
        # solve the model
        MOI.optimize!(model)
        
        # check status
        @assert MOI.get(model, MOI.TerminationStatus()) in [MOI.LOCALLY_SOLVED, MOI.OPTIMAL]
        
        # get and save the solution
        z = MOI.get(model, MOI.VariablePrimal(), var_idx)
        
        # get and save dual variables
        λ = MOI.get(model, MOI.ConstraintDual(), ineq_con_idx)
    
        if neq > 0
            ν = MOI.get(model, MOI.ConstraintDual(), eq_con_idx)
        end
    
        return z
    end
    
    """
        Method to differentiate and obtain gradients/jacobians
        of z, λ, ν  with respect to the parameters specified in
        in argument
    """
    function backward(params::Array{String})
        grads = Array{Float64}[]
        LHS = create_LHS_matrix(z, λ, Q, G, h, A)
        for param in params
            if param == "h"
                RHS = create_RHS_matrix(z, zeros(nz, nz), zeros(nz, 1), 
                                        λ, zeros(nineq, nz), ones(nineq,1),
                                        ν, zeros(neq, nz), zeros(neq, 1))
                push!(grads, LHS \ RHS)
            elseif param == "Q"
                RHS = create_RHS_matrix(z, ones(nz, nz), zeros(nz, 1),
                                        λ, zeros(nineq, nz), zeros(nineq,1),
                                        ν, zeros(neq, nz), zeros(neq, 1))
                push!(grads, LHS \ RHS)
            else
                push!(grads, [])
            end
        end
        return grads
    end
    
    () -> (forward, backward)
end

DiffOpt (generic function with 1 method)

In [31]:
using MathOptInterface
using Ipopt
using OSQP
using LinearAlgebra

const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities;

In [32]:
n = 2 # variable dimension
m = 6; # no of inequality constraints

In [33]:
# using example on https://osqp.org/docs/examples/setup-and-solve.html
Q = [4. 1.;1. 2.]
q = [1.; 1.]
G = [1. 1.;
    1. 0.;
    0. 1.;
    -1. -1.;
    -1. 0.;
    0. -1.]
h = [1.;
    0.7;
    0.7; 
    -1.;
    0.;
    0.];

In [34]:
# model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
# x = MOI.add_variables(model, n);

In [130]:
const IPO_OPTIMIZER = Ipopt.Optimizer(print_level=0)
MOI.set(IPO_OPTIMIZER, MOI.Silent(), true)
const IPO_CACHE = MOIU.UniversalFallback(MOIU.Model{Float64}())
model = MOIU.CachingOptimizer(IPO_CACHE, IPO_OPTIMIZER)

x = MOI.add_variables(model, n);

In [131]:
# define objective

quad_terms = MOI.ScalarQuadraticTerm{Float64}[]
for i in 1:n
    for j in i:n # indexes (i,j), (j,i) will be mirrored. specify only one kind
        push!(quad_terms, MOI.ScalarQuadraticTerm(Q[i,j],x[i],x[j]))
    end
end

objective_function = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm.(q, x),quad_terms,0.)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

MIN_SENSE::OptimizationSense = 0

In [132]:
# add constraints
for i in 1:m
    MOI.add_constraint(
        model,
        MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(G[i,:], x), 0.),
        MOI.LessThan(h[i])
    )
end

In [246]:
dd = DiffOpt(model)

#79 (generic function with 1 method)

In [247]:
pp = dd.forward()

2-element Array{Float64,1}:
 0.29999998486705753
 0.7000000054542964 

In [248]:
grads = dd.backward(["Q"])

1-element Array{Array{Float64,N} where N,1}:
 [-3.33746e-9; -3.04801e-16; … ; -3.37117e-17; -5.74678e-25]